In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# 2 환율 예측 프로젝트

### 1 데이터 확인하기

### 2 데이터 불러오기

1. 통계표 확인하기
2. 웹데이터 크롤링하기

환율 데이터 url 주소를 복사하여 가져오고, 판다스 모듈을 활용하여 데이터를 불러온다. 웹 크롤링을 활용할 수도 있지만, 이번 프로젝트에서는 판다스에서 제공해 주는 read_html() 함수를 활용하여 웹 데이터를 크롤링해 온다.

In [ ]:
import json

In [ ]:
import requests

In [ ]:
url = "https://api.odcloud.kr/api/15105540/v1/uddi:759b90cf-dbcb-4a1e-a269-be8faeb0c380?page=1&perPage=10&serviceKey=7NbYRS5ZqVR8B6kkCHNbn8viLQmZOi9V59miwtJUkIJkF0YUnW3EYldbufLYBuP1F3pA43MRb%2F4xUBIfDAvaHw%3D%3D"
res = requests.get(url)
text = res.text
mar_json = json.loads(text) #mar = market average rate

In [ ]:
mar_json

In [ ]:
print(json.dumps(mar_json,indent=4,sort_keys=True))

In [ ]:
print(mar_json.keys())

In [ ]:
print(mar_json['data'])

In [ ]:
item = mar_json['data']

In [ ]:
item

In [ ]:
for i, v in enumerate(mar_json['data']):
    print(i, v)

In [ ]:
for item in mar_json['data']:
    if '일자(년월일)' in item:
        print(item['일자(년월일)'])

In [ ]:
import pandas as pd
# url = 'https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1068&param=013'
url = './mar.csv'
data = pd.read_csv(url)
# data = pd.read_html(url)

In [ ]:
# data = pd.read_csv('mar.csv')
# data

### 3 데이터 전처리하기

크롤링한 웹 데이터에서 크롤링한 데이터를 불러와서 불필요한 데이터를 제거해 보자.

1. 데이터 불러오기

print(data)를 수행해 보면, 우리가 활용하고자 하는 정보는 두 번째에 담겨 있는 것을 확인 할 수 있다. 데이터의 두 번째 값을 불러온다.

In [ ]:
data

1. 불필요한 데이터 제거하기

우리는 원/달러 기준 환율을 분석하고자 하는 것이므로 두 번째 행을 제외하고 불필요한 데이터는 제거한다.

In [ ]:
dataFinal = data.iloc[0:2, 1:]
dataFinal

1. 칼럼명 변경하기

현재 칼럼값 중 모호한 값들을 변경해 주어야 한다. ‘Unnamed: 1’은 ‘구분’으로 대체해 주고, 나중에 이 값들을 회귀 분석헤 활용하기 위해 문자열이 들어 있는 값들은 모두 문자열을 제거해 줄 것이다.

In [ ]:
listTmp = list(dataFinal.columns)
listTmp[0] = '구분'
listTmp[4] = '202401'
listTmp[5] = '202402'
listTmp[6] = '202403'
listTmp[7] = '202404'
listTmp[8] = '202405'
listTmp[9] = '202406'
dataFinal.columns=listTmp
dataFinal

1. 인덱스 설정 및 행과 열 바꾸기

칼럼들을 인덱스로 만들고, 회귀 분석을 위해 데이터프레임의 행과 열의 축을 바꾼다.

In [ ]:
dataFinal = dataFinal.set_index('구분')
dataFinal

In [ ]:
dataFinal = dataFinal.T
dataFinal

### 4 데이터 시각화

데이터프레임 시각화를 위해 필요한 모듈을 불러오고, 한글 폰트가 깨지지 않게 해당 코드를 실행한 후, 코랩 상단 ‘런타임’ 메뉴에서 ‘런타임 다시 시작’을 클릭하여 코드를 재실행하자.

1. 한글 폰트 및 마이너스 글자 깨짐 처리하기

matplotlib에서 제목이나 x,y 축의 이름에 한글을 사용하면 한글이 깨지는 문제가 발생하므로 적절한 처리가 필요하다.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

1. 시각화하기

환율과 절상율에 따라 원/달러 추이 그래프를 시각화한다.

In [ ]:
# dataFinal.plot(figsize=(10,8))
# plt.title('원/달러 추이')
# plt.legend(loc='center left', bbox_to_anchor=(0.7, 0.5))
# plt.show()

### 5 회귀 분석 알고리즘 적용

현재 인덱스인 구분의 연도 데이터를 회귀 분석에 활용하기 위해 year라는 칼럼명으로 열을 추가하자.

In [ ]:
dataLR = dataFinal
dataLR['year'] = dataLR.index
dataLR

연도에 따른 환율을 계산할 것이므로 연도가 X값, 환율이 y값이 되도록 데이터를 분할한다.

In [ ]:
X = dataLR.iloc[:,-1:]
X

In [ ]:
y = dataLR.iloc[:,0]
y

1. 선형 회귀 알고리즘

선형 회귀 모델에 대한 훈련 데이터와 테스트 데이터의 예측 결과 점수를 확인해 보자.

In [ ]:
y = y.str.replace(',', '').astype(float)

from sklearn.linear_model import LinearRegression
# from sklearn.metrics import SCORES
from sklearn.model_selection import train_test_split

lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.3)
lr.fit(X_train, y_train)
print("훈련 세트 점수: {:.2f}".format(lr.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(lr.score(X_test, y_test)))

선형 회귀 알고리즘 테스트 결과, 훈련 데이터 점수 0.77인 반면 테스트 데이터 점수는 음수가 나오므로 선형 회귀는 유의미한 결과를 보이지 못한다.

1. KNN 회귀 알고리즘

k값에 따라 가장 정확도가 높은 k값을 설정하기 위해 반복문을 활용하여 k값의 변화에 따른 정확도를 출력한다.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

for k in range(1,7):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    print('train R^2 : {:.3f}'.format(knn.score(X_train, y_train)))
    print('test k : {}, R^2 : {:.3f}'.format(k, knn.score(X_test, y_test)))

훈련 데이터와 테스트 데이터 각각의 정확도를 살펴보려고 시각화할 수도 있다.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

training_accuracy=[]
test_accuracy=[]
neighbors=range(1,7)
for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    training_accuracy.append(knn.score(X_train, y_train))
    test_accuracy.append(knn.score(X_test, y_test))

plt.figure(figsize=(10,8))
plt.plot(neighbors, training_accuracy, label="Training Accuracy")
plt.plot(neighbors, test_accuracy, label="Test Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.xticks(neighbors)
plt.legend()

그 결과, k값이 1일 때 62%의 정확도를 보여 가장 정확도가 높았으므로 이를 토대로 모델을 만들어 2023년의 환율을 예측해 보자.

In [ ]:
knn = KNeighborsRegressor(n_neighbors=1)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

In [ ]:
knn.predict([[2025]]) # 2025년 환율 예측

2025년의 환율 예측 결과 1달러는 74% 확률로 1,288원이 될 것이다.
환율 데이터를 수집하여 회귀 분석 과정을 정리한 전체 소스 코드는 다음과 같다.